# Match statistics with player attributes
This generates the match statistics, but with player attributes and their position marked. The field is seperated in 8x22 grid. To keep things simple goalkeepers are excluded from data. Home players are even position of y, and away player are odd positions of y.

So imagining the field as a picture, then the player attributes are considered channels.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Grab dataframes for Match and Player_Attributes from Sqlite3 database.
conn = sqlite3.connect("database.sqlite")
df_match = pd.read_sql_query("select * from Match;", conn)
df_player = pd.read_sql_query("select * from Player_Attributes;", conn)
conn.close()

In [3]:
# Generate col names that have player position and api info
keep_cols = [col for col in df_match.columns.values if "_player_" in col]
keep_cols += ['stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']


# Generate cols that we want to drop (based on ones we want to keep)
drop_cols = [col for col in df_match.columns if col not in keep_cols]
# Drop goalkeeper info
drop_cols += ['home_player_1', 'away_player_1', 'home_player_X1', 'home_player_Y1', 'away_player_X1', 'away_player_Y1']

print(drop_cols)

['id', 'country_id', 'league_id', 'season', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'home_player_1', 'away_player_1', 'home_player_X1', 'home_player_Y1', 'away_player_X1', 'away_player_Y1']


In [4]:
# Drop columns that we generated above, do this before NaN check below
df_match.drop(drop_cols, axis=1, inplace=True)

# Cleanup dataset, drop any row with NaN values, reset indices
df_match.dropna(inplace=True)
df_match.reset_index(drop=True, inplace=True)
df_match.head()

,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X2,home_player_X3,home_player_X4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,24,2009-02-27 00:00:00,493017,8203,9987,2,1,2.0,4.0,6.0,...,38293.0,148313.0,104411.0,148314.0,37202.0,43158.0,9307.0,42153.0,32690.0,38782.0
1,24,2009-03-01 00:00:00,493019,9985,9984,4,0,2.0,4.0,6.0,...,37047.0,37021.0,38186.0,27110.0,32863.0,37957.0,37909.0,104386.0,38251.0,37065.0
2,25,2009-03-08 00:00:00,493025,9984,8342,1,3,2.0,4.0,6.0,...,21812.0,11736.0,37858.0,38366.0,37983.0,39578.0,38336.0,52280.0,27423.0,38440.0
3,25,2009-03-07 00:00:00,493027,8635,10000,2,0,2.0,4.0,6.0,...,37886.0,37903.0,37889.0,94030.0,37893.0,37981.0,131531.0,130027.0,38231.0,131530.0
4,26,2009-03-13 00:00:00,493034,8203,8635,2,1,2.0,4.0,6.0,...,38388.0,38389.0,31316.0,164694.0,30949.0,38378.0,38383.0,38393.0,38253.0,37069.0


In [5]:
# Drop goalkeeper stats, and stats related to set-piece (too situational to be useful)
drop_cols = [u'player_fifa_api_id', u'free_kick_accuracy', u'penalties', u'gk_diving', u'gk_handling', u'gk_kicking',
       u'gk_positioning', u'gk_reflexes']

# Drop columns that we generated above, do this before NaN check below
df_player.drop(drop_cols, axis=1, inplace=True)

# Cleanup dataset, drop any row with NaN values, reset indices
df_player.dropna(inplace=True)
df_player.reset_index(drop=True, inplace=True)
df_player.head()

,id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,...,stamina,strength,long_shots,aggression,interceptions,positioning,vision,marking,standing_tackle,sliding_tackle
0,1,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,76.0,35.0,71.0,70.0,45.0,54.0,65.0,69.0,69.0
1,2,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,76.0,35.0,71.0,70.0,45.0,54.0,65.0,69.0,69.0
2,3,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,...,54.0,76.0,35.0,63.0,41.0,45.0,54.0,65.0,66.0,69.0
3,4,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,...,54.0,76.0,34.0,62.0,40.0,44.0,53.0,62.0,63.0,66.0
4,5,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,...,54.0,76.0,34.0,62.0,40.0,44.0,53.0,62.0,63.0,66.0


## Check dataset validity
Now that we grabbed and cleaned our datasets, lets ensure that the api ids are correlated, namely all api id's used in df_match should also exist df_player. If not then drop that specific row

In [6]:
# Build the list of columns that store the api ids
player_api_cols = [col for col in df_match.columns if "_player_" in col and ("_X" not in col and "_Y" not in col )]
print(player_api_cols)

['home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']


In [7]:
print(df_match.shape)

(21490, 67)


In [8]:
for col_name in player_api_cols:
    comp_mask = np.isin(df_match[col_name], df_player['player_api_id'], invert=True)
    if comp_mask.any():
        print("Dropping %d" % len(df_match[comp_mask].index))
        df_match.drop(df_match[comp_mask].index, axis=0, inplace=True)

Dropping 432
Dropping 322
Dropping 253
Dropping 204
Dropping 159
Dropping 217
Dropping 130
Dropping 130
Dropping 128
Dropping 101
Dropping 239
Dropping 162
Dropping 149
Dropping 118
Dropping 89
Dropping 110
Dropping 79
Dropping 85
Dropping 66
Dropping 66


In [9]:
print(df_match.shape)

(18251, 67)


In [10]:
df_match.head()

,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X2,home_player_X3,home_player_X4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
3,25,2009-03-07 00:00:00,493027,8635,10000,2,0,2.0,4.0,6.0,...,37886.0,37903.0,37889.0,94030.0,37893.0,37981.0,131531.0,130027.0,38231.0,131530.0
4,26,2009-03-13 00:00:00,493034,8203,8635,2,1,2.0,4.0,6.0,...,38388.0,38389.0,31316.0,164694.0,30949.0,38378.0,38383.0,38393.0,38253.0,37069.0
9,29,2009-04-12 00:00:00,493061,8635,8342,1,0,2.0,4.0,6.0,...,39578.0,21812.0,11736.0,37858.0,37983.0,27364.0,38336.0,38366.0,27423.0,38440.0
13,32,2009-05-02 00:00:00,493089,10000,9985,0,0,2.0,4.0,6.0,...,39580.0,30692.0,38800.0,47411.0,35412.0,39631.0,39591.0,148335.0,37262.0,148315.0
20,34,2009-05-16 00:00:00,493107,9991,9985,0,1,2.0,4.0,6.0,...,30692.0,38800.0,37861.0,156551.0,35412.0,26224.0,39631.0,39591.0,37262.0,38369.0


In [11]:
# Delete positional info
pos_col = [col for col in df_match.columns if "_player_X" in col or "_player_Y" in col]
pos_col

['home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player_Y11']

In [12]:
df_match.drop(pos_col, axis=1, inplace=True)

In [13]:
df_match.head()

,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_2,home_player_3,home_player_4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
3,25,2009-03-07 00:00:00,493027,8635,10000,2,0,38388.0,26458.0,13423.0,...,37886.0,37903.0,37889.0,94030.0,37893.0,37981.0,131531.0,130027.0,38231.0,131530.0
4,26,2009-03-13 00:00:00,493034,8203,8635,2,1,67950.0,67958.0,38801.0,...,38388.0,38389.0,31316.0,164694.0,30949.0,38378.0,38383.0,38393.0,38253.0,37069.0
9,29,2009-04-12 00:00:00,493061,8635,8342,1,0,33620.0,38388.0,13423.0,...,39578.0,21812.0,11736.0,37858.0,37983.0,27364.0,38336.0,38366.0,27423.0,38440.0
13,32,2009-05-02 00:00:00,493089,10000,9985,0,0,37886.0,37100.0,37903.0,...,39580.0,30692.0,38800.0,47411.0,35412.0,39631.0,39591.0,148335.0,37262.0,148315.0
20,34,2009-05-16 00:00:00,493107,9991,9985,0,1,38337.0,38255.0,12473.0,...,30692.0,38800.0,37861.0,156551.0,35412.0,26224.0,39631.0,39591.0,37262.0,38369.0


## Done with match-field player api ids
We have now stored all the api ids. Just write to pickle so we can read from it later.

In [14]:
df_match.loc[:, 'date'] = pd.to_datetime(df_match['date'])

In [15]:
df_match.head()

,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_2,home_player_3,home_player_4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
3,25,2009-03-07,493027,8635,10000,2,0,38388.0,26458.0,13423.0,...,37886.0,37903.0,37889.0,94030.0,37893.0,37981.0,131531.0,130027.0,38231.0,131530.0
4,26,2009-03-13,493034,8203,8635,2,1,67950.0,67958.0,38801.0,...,38388.0,38389.0,31316.0,164694.0,30949.0,38378.0,38383.0,38393.0,38253.0,37069.0
9,29,2009-04-12,493061,8635,8342,1,0,33620.0,38388.0,13423.0,...,39578.0,21812.0,11736.0,37858.0,37983.0,27364.0,38336.0,38366.0,27423.0,38440.0
13,32,2009-05-02,493089,10000,9985,0,0,37886.0,37100.0,37903.0,...,39580.0,30692.0,38800.0,47411.0,35412.0,39631.0,39591.0,148335.0,37262.0,148315.0
20,34,2009-05-16,493107,9991,9985,0,1,38337.0,38255.0,12473.0,...,30692.0,38800.0,37861.0,156551.0,35412.0,26224.0,39631.0,39591.0,37262.0,38369.0


In [16]:
df_match.reset_index(drop=True, inplace=True)
df_match.to_pickle("match_field_plapi2.pkl")